In [231]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings('ignore')

In [232]:
from sklearn.model_selection import train_test_split, cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, f1_score, recall_score, precision_score,roc_auc_score, SCORERS, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [233]:
cancer = pd.read_csv('cancer_data.csv')
cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


# Analysis

* Goalnya ada lah mendeteksi cancer yang Malignant
* Kelas 0: Cancer Benign
* Kelas 1: Cancer Malignant

* FP : Memprediksi cancer yang malignant tapi actualnya Benign, impactnya orang tersebut akan stress dan kemungkinan mengeluarkan biaya tambahan
* FN : Memprediksi cancer yang Benign tapi actualnya Malignant, impactnya orang tersebut tidak akan sadar dengan kondisi cancer yang dimilikinya dan kemungkinan akan memperburuk kondisinya
* Jadi, dalam case ini kita akan menghindari FN dengan mendapatkan nilai Recall setinggi mungkin

# Preprocessing

In [234]:
cancer.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,0.0
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,NaN
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,NaN
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,NaN
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,NaN
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,NaN
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,NaN
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,NaN


In [235]:
cancer['diagnosis'].describe()

count     569
unique      2
top         B
freq      357
Name: diagnosis, dtype: object

In [236]:
cancer.isna().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [237]:
len(cancer)

569

* Terdapat missing value di seluruh row di column 'Unnamed: 32'
* Treatmentnya -> mendrop column 'Unnamed: 32' dan 'ID' karena tidak memiliki pengaruh apapun ke klasifikasi

In [238]:
cancer = cancer.drop(['id','Unnamed: 32'],axis=1)

In [239]:
cancer.isna().sum()

diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

In [240]:
cancer['diagnosis'] = np.where(cancer['diagnosis'].astype(str) =='B',0,1)
cancer

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [241]:
cancer['diagnosis'].unique()

array([1, 0])

* Mengubah kolom klasifikasi menjadi numerik agar dapat memakai beberapa model yang berbeda

# EDA

In [242]:
Q1 = cancer.quantile(0.25)
Q3 = cancer.quantile(0.75)
IQR = Q3 - Q1

In [243]:
((cancer < (Q1 - 1.5 * IQR)) | (cancer > (Q3 + 1.5 * IQR))).sum()/len(cancer)*100

diagnosis                   0.000000
radius_mean                 2.460457
texture_mean                1.230228
perimeter_mean              2.284710
area_mean                   4.393673
smoothness_mean             1.054482
compactness_mean            2.811951
concavity_mean              3.163445
concave points_mean         1.757469
symmetry_mean               2.636204
fractal_dimension_mean      2.636204
radius_se                   6.678383
texture_se                  3.514938
perimeter_se                6.678383
area_se                    11.423550
smoothness_se               5.272408
compactness_se              4.920914
concavity_se                3.866432
concave points_se           3.339192
symmetry_se                 4.745167
fractal_dimension_se        4.920914
radius_worst                2.987698
texture_worst               0.878735
perimeter_worst             2.636204
area_worst                  6.151142
smoothness_worst            1.230228
compactness_worst           2.811951
c

* Jika dilihat dari persentase outlier, untuk kolom area_se terdapat outlier lebih dari 10% dari keseluruhan datanya
* Angka tersebut lumayan tinggi oleh karena itu untuk scaling akan memakai robust scaler

In [258]:
cancer['diagnosis'].value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

* Datanya imbalance, jadi perlu stratify untuk class nya

# Splitting

In [262]:
X = cancer.drop(['diagnosis'],axis=1)
y = cancer['diagnosis']

In [263]:
X_train_val, X_test, y_train_val, y_test_val = train_test_split(X,y,stratify=y,test_size=0.2,random_state=2021)

# Transformer

In [264]:
transformer_knn = ColumnTransformer([
    ('scaler',RobustScaler(),X.columns)
    ],
    remainder='passthrough')
# transformer_logreg = ColumnTransformer([
#     ('scaler',RobustScaler(),['area_se'])
#     ],
#     remainder='passthrough')

# Model Benchmark

In [265]:
tree =  DecisionTreeClassifier(max_depth=5)
logreg = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier(n_neighbors=3)

In [266]:
#Karena dataset yang kecil Logistic Regression menggunakan liblinear

In [267]:
tree_pipe = Pipeline([
    ('tree',tree)
])

logreg_pipe = Pipeline([
    ('transformer',transformer_knn),
    ('logreg',logreg)
])

knn_pipe = Pipeline([
    ('transformer',transformer_knn),
    ('knn',knn)
])

In [268]:
def recallScore(pipeline):
    skfold = StratifiedKFold(n_splits=5)
    df = pd.DataFrame(columns=['Name','1','2','3','4','5','mean','std'])
    for i in pipeline:
        arr = cross_val_score(i, X_train_val, y_train_val, scoring='recall',cv=skfold)
        df = df.append({'Name':str(i[-1]), '1':arr[0], '2':arr[1], '3':arr[2], '4':arr[3], '5':arr[4], 'mean':arr.mean(),'std':arr.std()}, ignore_index=True)
    return df

In [269]:
pipe = [tree_pipe,logreg_pipe,knn_pipe]
recallScore(pipe)

,Name,1,2,3,4,5,mean,std
0,DecisionTreeClassifier(max_depth=5),0.941176,0.911765,0.970588,0.852941,0.911765,0.917647,0.039019
1,LogisticRegression(solver='liblinear'),0.970588,0.941176,1.000000,0.970588,0.911765,0.958824,0.029994
2,KNeighborsClassifier(n_neighbors=3),0.882353,0.911765,1.000000,0.911765,0.882353,0.917647,0.043226


* Berdasarkan benchmark tersebut, Semua model memiliki performa cross validation yang kurang baik, namun Logistic Regression memiliki mean recall tertinggi, oleh karena itu kita memilih Logistic Regression

## Model Performance with Test Set

In [270]:
logreg = LogisticRegression()
logreg_pipe = Pipeline([
    ('transformer',transformer_knn),
    ('logreg',logreg)
])


In [271]:
logreg_pipe.fit(X_train_val,y_train_val)
y_pred = logreg_pipe.predict(X_test)

In [272]:
print(classification_report(y_test_val,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99        72
           1       0.98      1.00      0.99        42

    accuracy                           0.99       114
   macro avg       0.99      0.99      0.99       114
weighted avg       0.99      0.99      0.99       114



In [273]:
recall_score(y_test_val,y_pred,pos_label=1)

1.0

* Karena recall score mencapai nilai maksimum maka kita tidak perlu melakukan tuning dan proses pemodelan sampai disini
* Classifier yang digunakan Logistic Regression menggunakan solver 'liblinear'

## FUNGSI EVALUASI

In [274]:
# def evaluation(pipeline,cv,metric):
#     skfold = StratifiedKFold(n_splits=5)
#     df = pd.DataFrame()
#     for i in pipeline:
#         arr = cross_val_score(i, X_train_val, y_train_val, scoring=metric,cv=cv)
#         if len(df) > 0
            
#             for i in temp:
#                 df
#         else:
#             temp = [x for i,j in enumerate(arr): str(i+1)]
#             df = pd.DataFrame(columns=['name']+temp+['mean','std'])
#     return df